In [254]:
import csv, json, math, re
from random import uniform
from shapely.geometry import shape, Point, Polygon

def LonLatToWebMercator(lon,lat):
    if lon != "" and lat != "":        
        lon = float(lon)
        lat = float(lat)
        if lat > 85.05113:
            lat = 85.05113
        if lat < -85.05113:
            lat = -85.05113
        x = (lon + 180.0) * 256.0 / 360.0
        y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    else:
        x = ""
        y = ""
    return [x, y]


In [240]:
# Convert CShapes (http://downloads.weidmann.ws/cshapes/Shapefiles/) Shapefile into GeoJSON
shpFilename = "cshapes/cshapes.shp"
geojsonFilename = "cshapes/cshaspes.geojson"
cmd = "ogr2ogr -f GeoJSON %s %s" % (geojsonFilename, shpFilename)
!$cmd

In [255]:
filename = "all.csv"
all_data = []
with open(filename) as f:
    reader = csv.DictReader(f)
    for row in reader:
        all_data.append(row)

In [256]:
filename = "gapminder.json"
with open(filename) as f:
    gapminder_data = json.load(f)

In [257]:
#Data missing from Gapminder
for country in all_data:
    alpha3 = country['alpha-3']
    name = country['name']
    match = False
    for row in gapminder_data['rows']:
        gapminder_geo = row[0].upper()
        gapminder_name = row[1]
        if gapminder_geo == alpha3 or gapminder_name == name:
            match = True
            break
    if not match:
        print "No match found for %s (%s) in Gapminder" % (country["name"], alpha3)

No match found for Bonaire, Sint Eustatius and Saba (BES) in Gapminder
No match found for French Southern Territories (ATF) in Gapminder
No match found for Heard Island and McDonald Islands (HMD) in Gapminder
No match found for Saint Barthélemy (BLM) in Gapminder
No match found for United States Minor Outlying Islands (UMI) in Gapminder


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:9: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if __name__ == '__main__':


In [258]:
#Data missing from ISO
for row in gapminder_data['rows']:
    gapminder_geo = row[0].upper()
    gapminder_name = row[1]
    for country in all_data:
        alpha3 = country['alpha-3']
        name = country['name']
        match = False
        if gapminder_geo == alpha3 or gapminder_name == name:
            match = True
            break
    if not match and row[2] != "":
        print "No match really found for %s in All" % row[1]


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:9: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if __name__ == '__main__':


In [259]:
all_data[0]

{'alpha-2': 'AF',
 'alpha-3': 'AFG',
 'country-code': '004',
 'iso_3166-2': 'ISO 3166-2:AF',
 'name': 'Afghanistan',
 'region': 'Asia',
 'region-code': '142',
 'sub-region': 'Southern Asia',
 'sub-region-code': '034'}

In [260]:
country_centroid_data = []
filename = "gothos/country_centroids_all.csv"
with open(filename) as csvfile:
    reader = csv.DictReader(csvfile, delimiter="\t")
    for row in reader:
        country_centroid_data.append(row)
#country_centroid_data[0]
#{'AFFIL': '',
# 'DMS_LAT': '330000',
# 'DMS_LONG': '660000',
# 'DSG': 'PCLI',
# 'FIPS10': 'AF',
# 'FULL_NAME': 'Islamic Republic of Afghanistan',
# 'ISO3136': 'AF',
# 'JOG': 'NI42-09',
# 'LAT': '33',
# 'LONG': '66',
# 'MGRS': '42STB1970055286',
# 'MOD_DATE': '2009-04-10',
# 'SHORT_NAME': 'Afghanistan'}


In [261]:
# Construct initial geojson
country_geojson = {
    "type": "FeatureCollections",
    "features": []
}

for country in all_data:
    alpha3 = country['alpha-3']
    num3 = country['country-code']
    name = country['name']
    match = False
    for row in gapminder_data['rows']:
        gapminder_geo = row[0].upper()
        gapminder_name = row[1]
        lng = row[3]
        lat = row[2]
        if gapminder_geo == alpha3 or gapminder_name == name:
            match = True            
            x,y = LonLatToWebMercator(lng, lat)        
            feature = {
                "type": "Feature",
                "geometry": {"type": "Point", "coordinates": [lng, lat]},
                "properties": {
                    "iso_alpha-3": alpha3,
                    "iso_num-3": num3,
                    "names": [country['alpha-2'], country['alpha-3'], country['country-code'], country['name'], gapminder_name, gapminder_geo.lower()], 
                    "webmercator": [x,y]
                    }
            }
            country_geojson["features"].append(feature)            
            break
    if not match:
        print "No match found for %s (%s) in Gapminder" % (country["name"], alpha3)
        with open("%s_adm0.geojson" % alpha3) as f:
            geojsonFile = json.load(f)
            polygon = shape(geojsonFile["features"][0]["geometry"])
            centroid = polygon.centroid.__geo_interface__['coordinates']
            x,y = LonLatToWebMercator(centroid[0], centroid[1])        
            feature = {
                "type": "Feature",
                "geometry": {"type": "Point", "coordinates": [centroid[0], centroid[1]]},
                "properties": {
                    "iso_alpha-3": alpha3,
                    "iso_num-3": num3,
                    "names": [country['alpha-2'], country['alpha-3'], country['country-code'], country['name']], 
                    "webmercator": [x,y]
                    }
            }
            country_geojson["features"].append(feature)            
        
# Add extra names to GeoJSON

for row in country_centroid_data:
    short_name = row["SHORT_NAME"]
    full_name = row["FULL_NAME"]
    iso3136 = row["ISO3136"]
    match = False
    for feature in country_geojson['features']:
        names = feature["properties"]["names"]
        for name in names:
            if short_name == name or iso3136 == name:
                match = True
                names.append(full_name)
                break
    if not match:
        print short_name
with open('gapminder.geojson', 'w') as f:
    json.dump(country_geojson, f, indent=4)    
        
with open('gapminder-min.geojson', 'w') as f:
    json.dump(country_geojson, f)    


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:17: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


No match found for Bonaire, Sint Eustatius and Saba (BES) in Gapminder
No match found for French Southern Territories (ATF) in Gapminder
No match found for Heard Island and McDonald Islands (HMD) in Gapminder
No match found for Saint Barthélemy (BLM) in Gapminder
No match found for United States Minor Outlying Islands (UMI) in Gapminder
Ashmore and Cartier Islands
Bassas da India
Clipperton Island
Coral Sea Islands
Europa Island
Glorioso Islands
Juan de Nova Island
Kosovo
Tromelin Island


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:61: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [262]:
# Add some name variants
# 1) Republic$ becomes Rep. or Rep
for feature in country_geojson["features"]:
    names = feature["properties"]["names"]
    for name in names:
        subnames = name.split(" ")
        if subnames[-1] == "Republic":
            names.append(name.replace("Republic", "Rep."))
            names.append(name.replace("Republic", "Rep"))
    feature["properties"]["names"] = list(set(names))
            
with open('gapminder.geojson', 'w') as f:
    json.dump(country_geojson, f, indent=4)    
        
with open('gapminder-min.geojson', 'w') as f:
    json.dump(country_geojson, f)    


In [263]:
# Add more name variants
# 2) Specifics from a CSV
variants = {
    (u"Côte d’Ivoire", "CIV"),
    ("Trinidad & Tobago", "TTO"),
    ("China, P.R.", "CHN"),
    ("Korea, North", "PRK"),
    ("Korea, South", "KOR"),
    ("Laos", "LAO"),
    ("Timor Leste", "TLS"),
    ("Bosnia-Herzegovina", "BIH"),
    ("UK", "GBR"),
    ("UAE", "ARE")
}
for variant in variants:
    for feature in country_geojson["features"]:
        name = feature["properties"]["iso_alpha-3"]
        if name == variant[1]:
            feature["properties"]["names"].append(variant[0])            
            feature["properties"]["names"] = list(set(feature["properties"]["names"]))
            break
            
with open('gapminder.geojson', 'w') as f:
    json.dump(country_geojson, f, indent=4)    
        
with open('gapminder-min.geojson', 'w') as f:
    json.dump(country_geojson, f)    


In [264]:
# Add more variants
# 3) historical variants 
with open("cshapes/cshapes.geojson") as f:
    cshapes_geojson = json.load(f)
    
variants = {
    ("Czechoslovakia", "Czechoslovakia", "200"),
    ("German DR", "Germany Democratic Republic", "278"),
    ("Germany (West)", "Germany Federal Republic", "280"),
    ("Kosovo", "Kosovo", "XKX"),
    ("Yugoslavia (former)", "Yugoslavia", "890"),
    ("USSR", "USSR", "810"),
    ("Yemen, North", "Yemen Arab Republic", "866"),
    ("Serbia and Montenegro", "Serbia and Montenegro", "891")
}

for variant in variants:
    for feature in cshapes_geojson["features"]:
        properties = feature["properties"]
        if properties["CNTRY_NAME"] == variant[1]:
            polygon = shape(feature["geometry"])
            centroid = polygon.centroid.__geo_interface__['coordinates']
            x,y = LonLatToWebMercator(centroid[0], centroid[1])        
            feature = {
                "type": "Feature",
                "geometry": {"type": "Point", "coordinates": [centroid[0], centroid[1]]},
                "properties": {
                    "iso_alpha-3": properties['ISO1AL3'] if properties['ISO1AL3'] is not None else "XKX",
                    "iso_num-3": variant[2],
                    "names": list(set([variant[2], variant[0], properties['CNTRY_NAME'], properties['ISONAME'] if properties['ISONAME'] is not None else "XKX", properties['ISO1AL3'] if properties['ISO1AL3'] is not None else "XKX"])), 
                    "webmercator": [x,y]
                    }
            }
            country_geojson["features"].append(feature)            
            break
            
with open('gapminder.geojson', 'w') as f:
    json.dump(country_geojson, f, indent=4)    
        
with open('gapminder-min.geojson', 'w') as f:
    json.dump(country_geojson, f)    


In [265]:
# More variants
# 4) Map "USSR/Russia" to Russia
variant = {"iso_num-3": "643", "name": "USSR/Russia"}
for feature in country_geojson["features"]:
    if feature["properties"]["iso_num-3"] == variant["iso_num-3"]:
        feature["properties"]["names"].append(variant["name"])

with open('gapminder.geojson', 'w') as f:
    json.dump(country_geojson, f, indent=4)    
        
with open('gapminder-min.geojson', 'w') as f:
    json.dump(country_geojson, f)    


In [266]:
# More variants
# 5) From UNHCR country list
variants = [
    {"name_en": "Curacao", "iso_num-3": "531"},
    {"name_en": "Federal Republic of Yugoslavia","iso_num-3": "890"},
    {"name_en": "Hong Kong SAR, China", "iso_num-3": "344"},
    {"name_en": "Kosovo (S/RES/1244 (1999))", "iso_num-3": "XKX"},
    {"name_en": "Libyan Arab Jamahiriya", "iso_num-3": "434"},
    {"name_en": "Macao SAR, China","iso_num-3": "446"},
    {"name_en": "Palestinian Territory, Occupied", "iso_num-3": "275"},
    {"name_en": "Serbia (and Kosovo: S/RES/1244 (1999))", "iso_num-3": "688"},
    {"name_en": "State of Palestine", "iso_num-3": "275"},
    {"name_en": "The former Yugoslav Republic of Macedonia", "iso_num-3": "807"},
    {"name_en": "Zaire", "iso_num-3": "180"}
]
    

for variant in variants:
    for feature in country_geojson["features"]:
        if feature["properties"]["iso_num-3"] == variant["iso_num-3"]:
            feature["properties"]["names"].append(variant["name_en"])
            break

with open('gapminder.geojson', 'w') as f:
    json.dump(country_geojson, f, indent=4)    
        
with open('gapminder-min.geojson', 'w') as f:
    json.dump(country_geojson, f)    


In [ ]:
len(cshapes_geojson["features"])

In [ ]:
cshapes_geojson["features"][231]['properties']

In [ ]:
i = 0
for feature in country_geojson['features']:
    gapminder_name = feature['properties']['names'][0]    
    match = False
    for row in country_data:
        short_name = row['SHORT_NAME']
        if short_name == gapminder_name:
            match = True
            country_geojson['features'][i]['properties']['names'].append(row['ISO3136'])             
            break
    if not match:
        print "Did not find %s" % gapminder_name
        i+= 1
i

In [ ]:
with open('gapminder.geojson', 'w') as f:
    json.dump(data, f, indent=4)


In [ ]:
filename = "ne_50m_admin_0_countries.geo.json"
with open(filename) as f:
    polygon_data = json.load(f)

In [ ]:
polygon_data['features'][1]['properties']

In [ ]:
for feature in polygon_data['features']:
    properties = feature['properties']
    if properties['name'] != properties['name_long'] or \
        properties['name'] != properties['name_sort'] or \
        properties['name_sort'] != properties['name_long']:
        print "MISMATCH %s %s %s" % (properties['name'], properties['name_long'], properties['name_sort'])

In [ ]:
country_geojson = {
    "type": "FeatureCollections",
    "features": []
}
filename = "country_polygons.geojson"
for feature in polygon_data['features']:
    geometry = feature['geometry']    
    properties = feature['properties']
    name_keys = ['abbrev','adm0_a3','admin','formal_en','iso_a2','iso_a3','iso_n3','name','name_long','name_sort']
    names = []
    for val in name_keys:
        if properties[val] is not None:
            names.append(properties[val])
    feature = {
        "type": "Feature",
        "geometry": geometry,
        "properties": { 'names': names}
        }
    country_geojson["features"].append(feature)
with open(filename, 'w') as f:
    json.dump(country_geojson, f, indent=4)


In [ ]:
for feature in data['features']:
    properties = feature['properties']
    names = properties['names']
    match = False
    for country in country_geojson['features']:
        country_names = country['properties']['names']
        for country_name in country_names:
            for name in names:
                if name == country_name:
                    match = True
                if match:
                    break
        if match:
            names += country_names
#            print "MATCH FOUND: " + names[0]
#            print "             " + ",".join(country_names)
            break

        

In [ ]:
with open('gapminder.geojson', 'w') as f:
    json.dump(data, f, indent=4)
